In [1]:
import $ivy.`org.flinkextended::flink-scala-api:1.17.1_1.1.0`
import $ivy.`org.apache.flink:flink-clients:1.17.1`
import $ivy.`org.apache.flink:flink-connector-files:1.17.1`
import $ivy.`org.apache.flink:flink-connector-kafka:1.17.1`

import org.apache.flinkx.api._
import org.apache.flinkx.api.serializers._

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import org.apache.flinkx.api._
import org.apache.flinkx.api.serializers._

In [2]:
val env = StreamExecutionEnvironment.getExecutionEnvironment

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


env: StreamExecutionEnvironment = org.apache.flinkx.api.StreamExecutionEnvironment@1e2e2e22

In [3]:
import org.apache.flink.connector.file.src.FileSource
import org.apache.flink.connector.file.src.reader.TextLineInputFormat
import org.apache.flink.core.fs.Path
import org.apache.flink.connector.file.src.impl.StreamFormatAdapter

val currentDirectory = new _root_.java.io.File(".").getCanonicalPath

val fileSource = FileSource.forBulkFileFormat(
    new StreamFormatAdapter(new TextLineInputFormat()), 
    new Path(s"$currentDirectory/sink-table")).build

import org.apache.flink.connector.file.src.FileSource
import org.apache.flink.connector.file.src.reader.TextLineInputFormat
import org.apache.flink.core.fs.Path
import org.apache.flink.connector.file.src.impl.StreamFormatAdapter
currentDirectory: String = "/Users/alexeyn/dev/git/flink-sandbox"
fileSource: FileSource[String] = org.apache.flink.connector.file.src.FileSource@2ede982d

In [3]:
import org.apache.flink.api.common.eventtime.WatermarkStrategy
env.fromSource(fileSource, WatermarkStrategy.noWatermarks(), "bootstrap-files").print()
env.execute()

-- [E006] Not Found Error: cell4.sc:2:13 ---------------------------------------
2 |val res4_1 = env.fromSource(fileSource, WatermarkStrategy.noWatermarks(), "bootstrap-files").print()
  |             ^^^
  |             Not found: env
  |
  | longer explanation available when compiling with `-explain`
-- [E006] Not Found Error: cell4.sc:3:13 ---------------------------------------
3 |val res4_2 = env.execute()
  |             ^^^
  |             Not found: env
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [3]:
import org.apache.flink.connector.kafka.source.KafkaSource
import org.apache.flink.connector.kafka.source.enumerator.initializer.OffsetsInitializer
import org.apache.flink.connector.base.source.hybrid.HybridSource
import org.apache.flink.connector.base.source.hybrid.HybridSourceSplitEnumerator
import org.apache.flink.connector.file.src.impl.StaticFileSplitEnumerator
import org.apache.flink.api.connector.source.Source
import org.apache.flink.api.connector.source.Boundedness
import org.apache.flink.api.common.serialization.SimpleStringSchema

val switchTimestamp = -1L
val brokers = "confluentkafka-cp-kafka:9092"
val kafkaSource = KafkaSource.builder[String]()
  .setBootstrapServers(brokers)
  .setTopics("bids")
  .setStartingOffsets(OffsetsInitializer.timestamp(switchTimestamp + 1))
  .setValueOnlyDeserializer(new SimpleStringSchema())
  .build()

val hybridSource = HybridSource
  .builder(fileSource)
  .addSource(kafkaSource)
  .build

-- [E006] Not Found Error: cell4.sc:20:11 --------------------------------------
20 |  .builder(fileSource)
   |           ^^^^^^^^^^
   |           Not found: fileSource
   |
   | longer explanation available when compiling with `-explain`
-- [E171] Type Error: cell4.sc:93:16 -------------------------------------------
88 |    _root_.ammonite
89 |          .repl
90 |          .ReplBridge
91 |          .value
92 |          .Internal
93 |          .print(brokers, "brokers", _root_.scala.None)
   |    ^
   |not enough arguments for method print in trait Internal: (implicit evidence$3: pprint.TPrint[String], tcolors: pprint.TPrintColors,
   |  classTagT: scala.reflect.ClassTag[String]): Iterator[String]
-- [E171] Type Error: cell4.sc:100:16 ------------------------------------------
 95 |    _root_.ammonite
 96 |          .repl
 97 |          .ReplBridge
 98 |          .value
 99 |          .Internal
100 |          .print(kafkaSource, "kafkaSource", _root_.scala.None)
    |    ^
    |not 

In [20]:
import org.apache.flink.api.common.eventtime.WatermarkStrategy

env.fromSource(hybridSource, WatermarkStrategy.noWatermarks(), "combined").print()
env.execute()

1> 3,3.4374177253612425E307,"2023-08-15 13:58:38.701"
1> 13,1.6315460424137705E308,"2023-08-15 13:58:38.729"
1> 23,8.632554082695326E307,"2023-08-15 13:58:38.73"
9> 4,7.257681906059148E306,"2023-08-15 13:58:38.701"
9> 14,7.316331551796105E307,"2023-08-15 13:58:38.73"
9> 24,4.0941573570201133E307,"2023-08-15 13:58:38.731"
9> 34,1.1207378234805113E308,"2023-08-15 13:58:38.731"
9> 44,7.989783066393725E307,"2023-08-15 13:58:38.731"
9> 54,6.441526317155994E307,"2023-08-15 13:58:38.732"
9> 64,1.6253137036132473E307,"2023-08-15 13:58:38.732"
9> 74,1.0948133384139185E308,"2023-08-15 13:58:38.732"
9> 84,1.5327140265641414E308,"2023-08-15 13:58:38.732"
9> 94,5.968415299898432E307,"2023-08-15 13:58:38.733"
9> 104,1.1895910545632384E308,"2023-08-15 13:58:38.733"
9> 114,4.663357915078976E307,"2023-08-15 13:58:38.733"
9> 124,7.435674895940279E307,"2023-08-15 13:58:38.733"
9> 134,1.7662312649770368E308,"2023-08-15 13:58:38.734"
9> 144,1.2287234875751214E308,"2023-08-15 13:58:38.734"
9> 154,7.04061890

Interrupted!: 

In [ ]:
val words = text
    .flatMap(_.toLowerCase.split("\\W+"))
    .map((_, 1))
    .keyBy(_._1)
    .sum(1)
    .executeAndCollect("wordCount").toList

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

val (x, y) = words.unzip

Bar(x, y).plot()